In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from IPython import get_ipython

# Define the URLs for the links
url1 = "/lab/"
url2 = "/voila/render/ui/conn.ipynb"
url3 = "/voila/render/ui/qprof_main.ipynb"

# Define the button labels
button_label1 = "Go to Home Page"
button_label2 = "Go to Connection"
button_label3 = "Go Back"

# Define HTML links for buttons
button_html1 = f"<a href='{url1}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px; margin-right: 10px;'><i class='fa fa-home'></i></a>"
button_html2 = f"<a href='{url2}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px;'>{button_label2}</a>"
button_html3 = f"<a href='{url3}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px; margin-right: 5px;'><i class='fa fa-arrow-left'></i></a>"


# Create HTML widgets
html_button1 = widgets.HTML(value=button_html1)
html_button2 = widgets.HTML(value=button_html2)
html_button3 = widgets.HTML(value=button_html3)


# Display buttons
display(widgets.HBox([html_button3, html_button1, html_button2]))


________

In [ ]:
import os
import logging

log_folder = 'logs'
if not os.path.exists(log_folder):
    os.makedirs(log_folder)
logging.basicConfig(filename='logs/jupyter_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
logging.info('[Query Profile Tree Page] Importing Libraries')
import pickle
import ipywidgets as widgets
from datetime import datetime, timezone
import time
import random
import string
import numpy as np
import verticapy as vp
import plotly.graph_objects as go
from collections import defaultdict, Counter
import textwrap
from verticapy.performance.vertica import QueryProfilerInterface, QueryProfiler
logging.info('[Query Profile Tree Page] Successfully Imported Libraries ')
output_initial = widgets.Output()
global qprof
global qprof2
with output_initial:
    %store -r query_val
    %store -r key_val
    %store -r target_schema_val
    %store -r transaction_val
    %store -r statement_val
    %store -r file_val
    %store -r target_schema_val_2
    %store -r key_val_2
    %store -r transaction_statement_list_val

    if "file_val" in globals():
        if not "key_val" in globals():
            characters = string.ascii_letters + string.digits  # includes 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
            key_val = ''.join(random.choice(characters) for _ in range(15))
        logging.info(f'[Query Profile Tree Page] Importing profile using target_schema as {target_schema_val}, key_id as {key_val}, filename as{file_val}')
        qprof = QueryProfiler.import_profile(target_schema=target_schema_val,
            key_id = key_val,
            filename=file_val,
            auto_initialize = False                                          
        )
        logging.info(f'[Query Profile Tree Page] Creating qprof object using target_schema as {target_schema_val}, key_id as {key_val if "key_val" in globals() else None}.')
        qprof = QueryProfilerInterface(
            target_schema = target_schema_val,
            key_id = key_val,
        )
    elif "target_schema_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using target_schema as {target_schema_val}, key_id as {key_val}. [Query Profile Tree Page]')
        qprof = QueryProfilerInterface(
            target_schema = target_schema_val,
            key_id = key_val,
        )
    elif "query_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using query as "{query_val}" target_schema as {target_schema_val_2 if "target_schema_val_2" in globals() else None}, key_id as {key_val_2 if "key_val_2" in globals() else None}')
        qprof = QueryProfilerInterface(
            query_val,
            target_schema = target_schema_val_2 if "target_schema_val_2" in globals() else None,
            key_id = key_val_2 if "key_val_2" in globals() else None,       
        )
    elif "transaction_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using transactions as {eval(transaction_val), eval(statement_val)}, target_schema as {target_schema_val_2 if "target_schema_val_2" in globals() else None}, key_id as{key_val_2 if "key_val_2" in globals() else None}')
        qprof = QueryProfilerInterface(
            transactions = (eval(transaction_val), eval(statement_val)),
            target_schema = target_schema_val_2 if "target_schema_val_2" in globals() else None,
            key_id = key_val_2 if "key_val_2" in globals() else None,
        )
    elif "transaction_statement_list_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using transactions as {eval(transaction_statement_list_val)}, target_schema as {target_schema_val_2 if "target_schema_val_2" in globals() else None}, key_id as{key_val_2 if "key_val_2" in globals() else None}')
        qprof = QueryProfilerInterface(
            transactions = eval(transaction_statement_list_val),
            target_schema = target_schema_val_2 if "target_schema_val_2" in globals() else None,
            key_id = key_val_2 if "key_val_2" in globals() else None,
        )


In [ ]:
# Loading display settings
with output_initial:
    # Attempt to load stored variables
    try:
        %store -r count_widget_val
    except KeyError:
        pass
    try:
        %store -r footer_widget_val
    except KeyError:
        pass
    try:
        %store -r comma_widget_val
    except KeyError:
        pass
    try:
        %store -r percent_bar_widget_val
    except KeyError:
        pass
    try:
        %store -r max_cell_width_widget_val
    except KeyError:
        pass
    try:
        %store -r max_columns_widget_val
    except KeyError:
        pass
    try:
        %store -r max_rows_widget_val
    except KeyError:
        pass

    # Set options using the loaded or default values
    try:
        if count_widget_val is not None:
            vp.set_option("count_on", count_widget_val)
        if footer_widget_val is not None:
            vp.set_option("footer_on", footer_widget_val)
        if comma_widget_val is not None:
            vp.set_option("insert_comma_numbers", comma_widget_val)
        if percent_bar_widget_val is not None:
            vp.set_option("percent_bar", percent_bar_widget_val)
        if max_cell_width_widget_val is not None:
            vp.set_option("max_cellwidth", max_cell_width_widget_val)
        if max_columns_widget_val is not None:
            vp.set_option("max_columns", max_columns_widget_val)
        if max_rows_widget_val is not None:
            vp.set_option("max_rows", max_rows_widget_val)
    except Exception as e:
        print(e)


In [ ]:
button_qprof_save = widgets.Button(description="Save Qprof Information & Report", layout=widgets.Layout(width='250px'))
output_export = widgets.Output()

def click_export_query(_):
    output_export.clear_output(wait=True)
    with output_export:
        print("Saving Query. It may take a while...")
        try:
            logging.info('[Query Profile Tree Page] Trying to export qprof object to a "qprof.tar" file')
            qprof.export_profile(filename="qprof.tar")
            print("Ready to Download")
            display(download_output)
            display(btn)
        except Exception as e:
            logging.info(f'[Query Profile Tree Page] Failed to export qprof because of this error:{e}')
            print("\033[91mUnsuccessful.\033[0m")
            print("Error:", e)
        print("\n Press the button below to generate and download the HTML report...")
        display(btn_report)


button_qprof_save.on_click(click_export_query)

qprof_save = widgets.VBox([button_qprof_save, output_export])

In [ ]:
import tarfile
from ipywidgets import Output, Button
from base64 import b64encode

download_output = Output()


def trigger_download(data, filename, file_type='application/octet-stream'):
    logging.info(f'[Query Profile Tree Page] Trying to download file')
    content_b64 = b64encode(data).decode()
    data_url = f'data:{file_type};base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click();
    """
    with download_output:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

btn = Button(description='Click to download the tar file', layout=widgets.Layout(width='250px'))

def download_qprof(e=None):
    # Assuming 'qprof.tar' is stored in the file path 'qprof_path'
    qprof_path = 'qprof.tar' 
    with open(qprof_path, 'rb') as file:
        qprof_data = file.read()
    trigger_download(qprof_data, 'qprof.tar')

btn.on_click(download_qprof)

In [ ]:
def trigger_download_report(data, filename, file_type='text/html'):
    logging.info(f'[Query Profile Tree Page] Trying to download HTML report.')
    content_b64 = b64encode(data).decode()
    data_url = f'data:{file_type};base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click();
    """
    with output_export:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

download_report_output = widgets.Output()
btn_report = Button(description='Click to download HTML report', layout=widgets.Layout(width='250px'))

def download_html(e=None):
    global qprof
    with output_export:
        print("Preparing files....")
        # Define the HTML content
        html_content = qprof.to_html()
        html_data = html_content.encode('utf-8')
        trigger_download_report(html_data, 'sample.html')      
        print("Done")


btn_report.on_click(download_html)

In [ ]:
top_accordion = widgets.Accordion(children=[qprof_save], titles=('Downloads',))
display(top_accordion)

In [ ]:
logging.info(f'[Query Profile Tree Page] Running the script to automatically display the tree')
from IPython.display import display,clear_output, IFrame, HTML
html_code = '''
<!DOCTYPE html>
<html>
<meta charset="utf-8">
<head>
    <link rel="modulepreload" href="https://d3js.org/d3.v5.min.js" >

</head>
<body>
    <div id="graph" style="text-align: center;"></div>
    <script src="https://d3js.org/d3.v5.min.js" type="module"></script>
    <script src="https://unpkg.com/@hpcc-js/wasm@0.3.11/dist/index.min.js"></script>
    <script src="https://unpkg.com/d3-graphviz@3.0.5/build/d3-graphviz.js"></script>
</body>
</html>
'''

In [ ]:
display(HTML(html_code))
logging.info(f'[Query Profile Tree Page] Completed the script to automatically display the tree')

In [ ]:
try:
    transaction_statement_list = qprof.transactions
    logging.info(f'[Query Profile Tree Page] The list of transaction and statement ids are: {transaction_statement_list}')
    # Construct the WHERE clause dynamically
    where_clause = " OR ".join([f"(transaction_id = {elem[0]} AND statement_id = {elem[1]})" for elem in transaction_statement_list])
    
    # Construct the query
    query = f"""
    SELECT transaction_id, statement_id, node_name, event_description, time, duration_us/1000000 AS duration_sec
    FROM dc_slow_events
    WHERE {where_clause}
    ORDER BY duration_sec DESC
    LIMIT 500;
    """
    query_2 = f"""
    SELECT *
    FROM dc_optimizer_events
    WHERE {where_clause};
    """
    logging.info(f'[Query Profile Tree Page] Trying to get the DC SLOW EVENTS table')
    res_1 = vp.vDataFrame(f"""{query}""")
    res_2 = vp.vDataFrame(f"""{query_2}""")
    table_1 = widgets.Output()
    table_2 = widgets.Output()
    with table_1:
        res_1.idisplay()
    with table_2:
        res_2.idisplay()
    tables = widgets.Tab()
    tables.children = [table_1, table_2]
    tables.titles = ["Slow Events", "Optimimzer Events"]

except Exception as e:
    tables = widgets.VBox([widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")])


button_get_queries = widgets.Button(description="Get Detailed Info")
output_get_queries = widgets.Output()

def on_button_clicked_get_queries(b):
    output_get_queries.clear_output(wait=True)
    with output_get_queries:
        try:
            vdf = qprof.get_queries()[qprof.get_queries().get_columns(exclude_columns = "is_current")]
            vdf["qduration"].rename("duration (seconds)")
            vdf.idisplay()
        except Exception as e:
            widgets.VBox([widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")])

button_get_queries.on_click(on_button_clicked_get_queries)

In [ ]:
# Query Engine Table tab
engine_tab_summary_1 = widgets.Output()

# List of checkbox tags
tags = [
    "exec_time_us", "est_rows", "proc_rows", "prod_rows", "rle_prod_rows",
    "blocks_filtered_sip", "blocks_analyzed_sip", "container_rows_filtered_sip",
    "container_rows_filtered_pred", "container_rows_pruned_sip"
]

# Create checkboxes
checkboxes = [
    widgets.Checkbox(value=(tag in ["exec_time_us", "est_rows", "rle_prod_rows"]), description=tag) 
    for tag in tags
]
selected_checkboxes = []
# Function to update based on selections
def on_checkbox_change(change):
    global selected_checkboxes, detailed_vdf
    selected_checkboxes = [cb.description for cb in checkboxes if cb.value]
    if len(selected_checkboxes) > 6:
        # Deselect the latest selection if limit is exceeded
        change['owner'].value = False
        return

    # Update the dataframe based on selected checkboxes
    coloumn_list = ["node_name", "path_id", "localplan_id", "operator_name"]
    coloumn_list.extend(selected_checkboxes.copy())
    detailed_vdf = qprof._get_vdf_summary()[0][coloumn_list]
    
    # Update the table display
    detailed_update_table(node_name=detailed_node_name_dropdown.value, path_id=detailed_path_id_dropdown.value, localplan_id=detailed_localplan_id_dropdown.value)


# Attach the observer to each checkbox
for cb in checkboxes:
    cb.observe(on_checkbox_change, names='value')

try:
    detailed_vdf = qprof._get_vdf_summary()[0]["node_name", "path_id", "localplan_id", "operator_name", "exec_time_us", "est_rows", "rle_prod_rows"]

    def detailed_update_table(node_name = "All", path_id = "All", localplan_id = "All", opeartor_name = "All"):
        vdf=detailed_vdf
        with engine_tab_summary_1:
            engine_tab_summary_1.clear_output(wait=True)
            if not path_id=="All":
                vdf=vdf[vdf["path_id"]==path_id]
            if not node_name=="All":
                vdf=vdf[vdf["node_name"]==node_name]
            if not localplan_id=="All":
                vdf=vdf[vdf["localplan_id"]==localplan_id]
            if not opeartor_name=="All":
                vdf=vdf[vdf["opeartor_name"]==opeartor_name]
            display(widgets.HTML(vdf._repr_html_()))
    detailed_update_table(node_name = "All", path_id = "All", localplan_id = "All", opeartor_name = "All")
    


    # Nodename drop down
    detailed_node_name_dropdown = widgets.Dropdown(
        options=['All'] + sorted(detailed_vdf["node_name"].distinct()),
        description='Select Node Name:',
        style={'description_width': 'initial'},
        value = 'All'
    )
    # Path ID dropdown
    detailed_path_id_dropdown = widgets.Dropdown(
        options=['All'] + sorted(detailed_vdf["path_id"].distinct()),
        description='Select Path ID:',
        style={'description_width': 'initial'},
        value = 'All'
    )
    # Local Plan ID dropdown
    detailed_localplan_id_dropdown = widgets.Dropdown(
        options=['All'] + sorted(detailed_vdf["localplan_id"].distinct()),
        description='Select Local Plan ID:',
        style={'description_width': 'initial'},
        value = 'All'
    )

    engine_table_summary_controls = widgets.VBox([detailed_node_name_dropdown, detailed_path_id_dropdown, detailed_localplan_id_dropdown])
    
    def summary_on_node_name_dropdown_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected_key = change['new']
            if selected_key == 'All':
                path_id_dropdown.options = ['All'] + sorted(detailed_vdf["path_id"].distinct())
            else:
                vdf = detailed_vdf[detailed_vdf["node_name"]==selected_key]
                detailed_path_id_dropdown.options = ['All'] + sorted(vdf["path_id"].distinct())
            detailed_path_id_dropdown.value = 'All'
            detailed_update_table(node_name = selected_key)
    
    detailed_node_name_dropdown.observe(summary_on_node_name_dropdown_change, names='value')
    
    def summary_on_path_id_dropdown_change(change):
        if not detailed_node_name_dropdown.value == "All":
            vdf=detailed_vdf[detailed_vdf["node_name"]==detailed_node_name_dropdown.value]
        else:
            vdf = detailed_vdf
        if change['type'] == 'change' and change['name'] == 'value':
            selected_key = change['new']
            if selected_key == 'All':
                detailed_localplan_id_dropdown.options = ['All'] + sorted(vdf["localplan_id"].distinct())
            else:
                vdf = vdf[detailed_vdf["path_id"]==selected_key]
                detailed_localplan_id_dropdown.options = ['All'] + sorted(vdf["localplan_id"].distinct())
            detailed_localplan_id_dropdown.value = 'All'
            detailed_update_table(node_name = detailed_node_name_dropdown.value,path_id = selected_key)
            
    detailed_path_id_dropdown.observe(summary_on_path_id_dropdown_change, names='value')
    
    def summary_on_localplan_id_dropdown_change(change):
        if not detailed_node_name_dropdown.value == "All":
            vdf=detailed_vdf[detailed_vdf["node_name"]==detailed_node_name_dropdown.value]
        else:
            vdf = detailed_vdf
        if not detailed_path_id_dropdown.value == "All":
            vdf=vdf[vdf["path_id"]==detailed_path_id_dropdown.value]
        if change['type'] == 'change' and change['name'] == 'value':
            selected_key = change['new']
            # if selected_key == 'All':
            #     localplan_id_dropdown.options = ['All'] + sorted(vdf["localplan_id"].distinct())
            # else:
            #     vdf = vdf[detailed_vdf["path_id"]==selected_key]
            #     localplan_id_dropdown.options = ['All'] + sorted(vdf["localplan_id"].distinct())
            # localplan_id_dropdown.value = 'All'
            detailed_update_table(node_name = detailed_node_name_dropdown.value,path_id = detailed_path_id_dropdown.value,localplan_id = selected_key)
    
    detailed_localplan_id_dropdown.observe(summary_on_localplan_id_dropdown_change, names='value')   
    # Organize the layout
    # engine_tab_summary_left = widgets.VBox([engine_table_summary_controls, engine_tab_summary_1])
    # checkbox_output = widgets.Output()
    # with checkbox_output:
    #     display(*checkboxes)
    # all_checkboxes = widgets.VBox([
    #     widgets.HTML('<h3> Select Metrics </h3>'),
    #     checkbox_output
    # ],
    #                              )
    
    # engine_tab_summary_all = widgets.HBox([engine_tab_summary_left , all_checkboxes])



    grid = widgets.GridspecLayout(2, 5, 
                                  # height='600px', 
                                  # width='100%'
                                 )

    # Add widgets to the grid
    grid[0:1, 0:4] = widgets.VBox([engine_table_summary_controls, engine_tab_summary_1])
    grid[0:2, 4] = widgets.VBox([widgets.HTML('<div style="text-align: center;"><h3>Select Metrics</h3></div>')] + checkboxes)

    engine_tab_summary_all = grid



except Exception as e:
    engine_tab_summary_all = widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")

engine_table_tab = engine_tab_summary_all

In [ ]:
# Plots
# query_execution_steps_widget = qprof.get_qsteps()
try:
    query_plan_profile = qprof.get_qplan_profile()
    query_plan_profile_plot = go.FigureWidget(data=[query_plan_profile.data[i] for i in range(len(query_plan_profile.data))],)
    
    query_execution = qprof.get_qexecution()
    query_execution_plot = go.FigureWidget(data=[query_execution.data[i] for i in range(len(query_execution.data))],)
    
    plot_tabs = widgets.Tab()
    
    plot_tabs.children = [query_plan_profile_plot, query_execution_plot]
    plot_tabs.titles = ["Query Plan Profile", "Query Execution Plot"]
except Exception as e:
    plot_tabs = widgets.VBox([widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")])

In [ ]:
# Tab for Query Statistics

button_query_stats = widgets.Button(description="Run statistics")
output_query_stats = widgets.Output()

# Display charts using widgets
output_optimize_stat = widgets.Output()
output_network_stat = widgets.Output()
output_execution_stat = widgets.Output()

vertical_line = widgets.HTML(value='<div style="border-left: 1px solid #F2F1EF; height: 100%;"></div>')
vertical_line_gap = widgets.HTML(value='<div style="border-left: 1px solid white; height: 100%;"></div>')

apply_button = widgets.Button(
    description='APPLY',
    button_style='success',  # 'success' (green), 'info' (light blue), 'warning' (yellow), 'danger' (red)
    tooltip='Apply filters',
    icon='check'  # (FontAwesome names without the `fa-` prefix)
)

def update_table_widget_professional(data, node_filter, category_filter):
    # Filter data based on selected node and category
    filtered_data = []
    for priority_list in data:
        filtered_priority_list = [item for item in priority_list 
                                  if (node_filter == 'All' or item[0] == node_filter) and 
                                     (category_filter == 'All' or item[1] == category_filter)]
        filtered_data.append(filtered_priority_list)
    return create_table_widget_professional(tuple(filtered_data))
all_tests = None
node_dropdown = None
category_dropdown = None
def on_button_clicked_query_stats(b):
    global all_tests, node_dropdown, category_dropdown
    button_query_stats.disabled = True
    output_query_stats.clear_output(wait=True)
    try:
        all_tests = qprof.main_tests()
        node_dropdown = widgets.Dropdown(description="Node", options=["All"] + sorted({val[0] for test in all_tests for val in test}), value = 'All')
        category_dropdown = widgets.Dropdown(description="Category", options=["All"] + sorted({val[1] for test in all_tests for val in test}), value = 'All')

        with output_query_stats:           
            optimize_chart = create_pie_chart("OPTIMIZER",analyze_stats_data(all_tests)[0])
            network_chart = create_pie_chart("NETWORK",analyze_stats_data(all_tests)[1])
            execution_chart = create_pie_chart("EXECUTION",analyze_stats_data(all_tests)[2])
            
            with output_optimize_stat:
                display(optimize_chart)
            
            with output_network_stat:
                display(network_chart)
            
            with output_execution_stat:
                display(execution_chart)
            display(widgets.HTML(value="<hr>"))
            display(widgets.HTML(value="<h1> Overall stats </h1>"))
            
            # Arrange charts horizontally
            display(widgets.HBox([output_optimize_stat,vertical_line_gap,vertical_line, output_network_stat,vertical_line_gap, vertical_line, output_execution_stat]))

            display(widgets.HTML(value="<hr>"))
            display(widgets.HTML(value="<h1> Node stats </h1>"))
            filter_title = widgets.HTML(
                                value='<h2>Filters</h2>'
                            )
            filters_box = widgets.VBox([filter_title, node_dropdown, category_dropdown,widgets.HTML("<br>"), apply_button], 
                                       layout=widgets.Layout(
                                           border='1px solid #ddd', 
                                           padding='10px', 
                                           border_radius='5px', 
                                           box_shadow='0 2px 5px rgba(0,0,0,0.1)'
                                       ))
            display(filters_box)
            node_filter = node_dropdown.value
            category_filter = category_dropdown.value
            with out_apply_button:
                display(update_table_widget_professional(all_tests, node_dropdown.value, category_dropdown.value))
            # filtered_table_widget = update_table_widget_professional(all_tests, node_filter, category_filter)
            # display(filtered_table_widget)
            # tests_table_widget = create_table_widget_professional(all_tests)
            # display(tests_table_widget)
    except Exception as e:
        widgets.VBox([widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")])
    button_query_stats.disabled = False

out_apply_button = widgets.Output()
def on_button_clicked_apply_button(b):
    out_apply_button.clear_output(wait=True)
    with out_apply_button:
        display(update_table_widget_professional(all_tests, node_dropdown.value, category_dropdown.value))

apply_button.on_click(on_button_clicked_apply_button)
button_query_stats.on_click(on_button_clicked_query_stats)


# Constants for column visibility
SHOW_NODE = True
SHOW_CATEGORY = True
SHOW_TYPE = True
SHOW_DESCRIPTION = True
SHOW_REMEDY = True
SHOW_ACTION = True

# Column widths (adjust as needed)
COLUMN_WIDTHS = {
    'NODE': '15%',
    'CATEGORY': '10%',
    'TYPE': '20%',
    'DESCRIPTION': '20%',
    'REMEDY': '20%',
    'ACTION': '15%'
}

def wrap_text(text, max_width=45):
    return '<br>'.join(textwrap.wrap(text, max_width))

def create_table_widget_professional(input_tuple):
    style = {'description_width': 'initial'}
    layout = widgets.Layout(width='100%')
    
    # Custom CSS for the entire table and header
    custom_css = """
    <style>
    .professional-table {
        font-family: Arial, sans-serif;
        border-collapse: collapse;
        width: 100%;
        border: 1px solid #ddd;
        border-radius: 8px;
        overflow: hidden;
    }
    .professional-header {
        background: linear-gradient(to bottom, #f8f8f8 0%, #e8e8e8 100%);
        color: #333;
        font-weight: bold;
        text-align: left;
        padding: 12px 15px;
        border-bottom: 2px solid #ddd;
    }
    .professional-header th {
        padding: 12px 15px;
    }
    .professional-row {
        transition: background-color 0.3s;
    }
    .professional-row:hover {
        filter: brightness(95%);
    }
    .professional-cell {
        padding: 12px 15px;
        border-bottom: 1px solid #ddd;
        word-wrap: break-word;
        overflow-wrap: break-word;
        white-space: normal;
    }
    .professional-button {
        border: none;
        color: white;
        text-align: center;
        text-decoration: none;
        display: inline-block;
        font-size: 14px;
        margin: 4px 2px;
        transition-duration: 0.4s;
        cursor: pointer;
        border-radius: 4px;
    }
    .professional_button_enabled:hover {
        background-color: #45a049;
    }
    .professional_button_enabled {
        background-color: #32cee6;
    }
    .professional_button_disabled {
        background-color: #d4d8d9;
    }
    .professional_button_disabled:hover {
        background-color: #d4d8d9;
    }
    </style>
    """
    display(HTML(custom_css))
    
    # Dynamically create header based on visible columns
    header_columns = []
    visible_columns = []
    if SHOW_NODE: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['NODE']}'>Node</th>")
        visible_columns.append('NODE')
    if SHOW_CATEGORY: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['CATEGORY']}'>Category</th>")
        visible_columns.append('CATEGORY')
    if SHOW_TYPE: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['TYPE']}'>Type</th>")
        visible_columns.append('TYPE')
    if SHOW_DESCRIPTION: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['DESCRIPTION']}'>Description</th>")
        visible_columns.append('DESCRIPTION')
    if SHOW_REMEDY: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['REMEDY']}'>Remedy</th>")
        visible_columns.append('REMEDY')
    if SHOW_ACTION: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['ACTION']}'>Action</th>")
        visible_columns.append('ACTION')
    
    header = widgets.HTML(
        value=f"""
        <table class="professional-table">
            <thead>
                <tr class="professional-header">
                    {"".join(header_columns)}
                </tr>
            </thead>
        </table>
        """
    )

    row_widgets = []
    input_tuple = input_tuple[::-1]
    for priority, rows in enumerate(input_tuple):
        if priority == 0:
            bg_color = '#ffcccb'  # light red/pink for high priority
        elif priority == 1:
            bg_color = '#ffd7b5'  # light orange for medium priority
        else:
            bg_color = '#c7f2c2' #'#f0f0f0'  # neutral gray for low priority

        for i, row in enumerate(rows):
            node, category, type_, description, remedy, action = row + [''] * (6 - len(row))  # Pad with empty strings if needed
            
            row_cells = []
            for column in visible_columns:
                if column == 'NODE':
                    row_cells.append(widgets.HTML(value=f'<div style="background-color: {bg_color}; width: 100%; height: 100%;"><div class="professional-cell">{wrap_text(node)}</div></div>', layout=widgets.Layout(width=COLUMN_WIDTHS['NODE'])))
                elif column == 'CATEGORY':
                    row_cells.append(widgets.HTML(value=f'<div style="background-color: {bg_color}; width: 100%; height: 100%;"><div class="professional-cell">{wrap_text(category)}</div></div>', layout=widgets.Layout(width=COLUMN_WIDTHS['CATEGORY'])))
                elif column == 'TYPE':
                    row_cells.append(widgets.HTML(value=f'<div style="background-color: {bg_color}; width: 100%; height: 100%;"><div class="professional-cell">{wrap_text(type_)}</div></div>', layout=widgets.Layout(width=COLUMN_WIDTHS['TYPE'])))
                elif column == 'DESCRIPTION':
                    row_cells.append(widgets.HTML(value=f'<div style="background-color: {bg_color}; width: 100%; height: 100%;"><div class="professional-cell">{wrap_text(description)}</div></div>', layout=widgets.Layout(width=COLUMN_WIDTHS['DESCRIPTION'])))
                elif column == 'REMEDY':
                    row_cells.append(widgets.HTML(value=f'<div style="background-color: {bg_color}; width: 100%; height: 100%;"><div class="professional-cell">{wrap_text(remedy)}</div></div>', layout=widgets.Layout(width=COLUMN_WIDTHS['REMEDY'])))
                elif column == 'ACTION':
                    remedy_button = widgets.Button(
                        description="Take Action" if action else "No Action",
                        disabled = True if action=="" else False,
                        layout=widgets.Layout(width='100%', height = '40px'),
                        style=style
                    )
                    remedy_button.add_class('professional-button')
                    if action:
                        remedy_button.add_class('professional_button_enabled')
                    else:
                        remedy_button.add_class('professional_button_disabled')
                    remedy_button.on_click(lambda b, action=action: remedy_action(action)(b))
                    row_cells.append(widgets.HBox([remedy_button], layout=widgets.Layout(width=COLUMN_WIDTHS['ACTION'], justify_content='center', align_items='center', background_color=bg_color)))
            
            row = widgets.HBox(row_cells, layout=widgets.Layout(margin='0', padding='0', width='100%'))
            row_widgets.append(row)

    table = widgets.VBox([header] + row_widgets, 
                         layout=widgets.Layout(margin='20px 0',
                                               box_shadow='0 0 10px rgba(0,0,0,0.1)'))
    
    return table



def remedy_action(sql_action):
    def action_handler(b):
        ip = get_ipython()
        
        if sql_action:
            ip.user_ns['sql_action'] = sql_action  # Add the variable to the user namespace
            ip.run_line_magic('store', 'sql_action')  # Store the variable
        else:
            ip.run_line_magic('store', '-d sql_action')  # Delete the stored variable
        with out_apply_button:
            notebook_url = '/voila/render/ui/sql_action.ipynb'
            display(HTML(f"<script>window.open('{notebook_url}');</script>"))
    
    return action_handler


def parse_tuple(data_tuple):
    # Define the priorities for each list
    priorities = [1, 2, 3]

    # Initialize the result list
    result = []

    # Loop through each list in the tuple and its corresponding priority
    for priority, data_list in zip(priorities, data_tuple):
        for item in data_list:
            # Append the type and priority to the result list
            result.append([item[1], priority])

    return result

def analyze_stats_data(data):
    # Initialize counters for each category
    counts = {
        "OPTIMIZATION": [0, 0, 0],  # counts for low, medium, and high priority
        "NETWORK": [0, 0, 0],
        "EXECUTION": [0, 0, 0]
    }
    
    # Iterate over each list (low, medium, high priority)
    for priority_level, priority_data in enumerate(data):
        for item in priority_data:
            if len(item) > 1:
                category = item[1]
                if category in counts:
                    counts[category][priority_level] += 1
    
    # Extract counts for each category
    optimization_alerts = counts["OPTIMIZATION"]
    network_alerts = counts["NETWORK"]
    execution_alerts = counts["EXECUTION"]
    
    return optimization_alerts, network_alerts, execution_alerts




def create_pie_chart(title,values):
    total = sum(values)
    
    # Avoid division by zero if total is 0
    if total == 0:
        raise ValueError("The sum of the numbers cannot be 0.")
    
    # Calculate percentages
    percentage1 = (values[0] / total) * 100
    percentage2 = (values[1] / total) * 100
    percentage3 = (values[2] / total) * 100

    percentages = [percentage1, percentage2, percentage3]
    labels = ['Low', 'Medium', 'High']
    color_sequence = ['green', 'orange', 'red']
    
    # Calculate the cumulative sum and angles
    cumsum = np.cumsum(percentages)
    total_percentage = sum(percentages)
    angles = (cumsum / total_percentage) * 180
    
    # Create the arch chart
    fig = go.Figure()
    
    for i in range(len(values)):
        start_angle = angles[i-1] if i > 0 else 0
        end_angle = angles[i]
        
        fig.add_trace(go.Barpolar(
            r=[0.6],
            theta=[start_angle + (end_angle - start_angle)/2],
            width=[end_angle - start_angle],
            base=0.4,
            marker_color=color_sequence[i % len(color_sequence)],
            name=f"{labels[i]} ({percentages[i]:.2f}%)",
            text=[f"{labels[i]}<br>No. of Alerts: {values[i]}"],
            # textinfo="text",  # Use textinfo instead of textposition
            hoverinfo="text"
        ))
    
    # Customize the layout
    fig.update_layout(
        title=dict(
                    text=title,
                    x=0.5,
                    xanchor='center'
                ),
        polar=dict(
            radialaxis=dict(
                range=[0, 1], 
                showticklabels=False, 
                ticks='',
                showline=False,
                showgrid=False
            ),
            angularaxis=dict(
                showticklabels=False, 
                ticks='',
                showline=False,
                showgrid=False
            ),
            sector=[0, 180],
            bgcolor='rgba(0,0,0,0)'  # Transparent background
        ),
        showlegend=True,
        width=700,
        height=500,
        paper_bgcolor='rgba(0,0,0,0)',  # Transparent paper background
        plot_bgcolor='rgba(0,0,0,0)'    # Transparent plot background
    )
    return fig

try:
    drop_down_query_select = qprof.query_select_dropdown
except Exception as e:
    drop_down_query_select = widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")

In [ ]:
output_tree = widgets.Output()

with output_tree:
    logging.info(f'[Query Profile Tree Page] Trying to display the tree')
    qprof.get_qplan_tree()
    logging.info(f'[Query Profile Tree Page] Finished trying to display the tree')
    
tabs = widgets.Tab()

child_1 = widgets.VBox([output_tree])
child_2 = widgets.VBox([drop_down_query_select, button_query_stats, output_query_stats, out_apply_button])
child_3 = engine_table_tab
child_4 = widgets.VBox([button_get_queries, output_get_queries])
child_5 = widgets.VBox([tables])
child_6 = widgets.VBox([plot_tabs])

tabs.children = [child_1, 
                 child_2, 
                 child_3, 
                 child_4, 
                 child_5, 
                 child_6]
tabs.titles = ["Tree", 
               "Query Statistics", 
               "Query Engine", 
               "Transaction List", 
               "Other info", 
               "Plots"]

# Define the observer function
def on_tab_change(change):
    if change['new'] == 2:  # Check if the first tab (index 0) is selected
        try:
            update_table('All', 'All')
        except Excetion as e:
            pass
        # Stop observing after the first time
        tabs.unobserve(on_tab_change, names='selected_index')

# Attach the observer
tabs.observe(on_tab_change, names='selected_index')

display(tabs)

In [ ]:
query = widgets.Textarea(
    placeholder='Enter a SQL Query',
    description='SQL Query:',
    disabled=False,
    layout=widgets.Layout(width='700px')
)
button_run_query = widgets.Button(description="Run Query")
output_run_query = widgets.Output()
query_val = None

def on_button_clicked_run_query(b):
    output_run_query.clear_output(wait=True)
    global query_val, target_schema_val_2, key_val_2
    with output_run_query:
        query_val = query.value
        logging.info(f"""[Query Profile Tree Page] Running the following query manually :{query_val}.""")
        now = datetime.now(timezone.utc).astimezone()
        current_time = now.strftime("%Y-%m-%d %H:%M:%S %Z %z")
        display(HTML(f"<b>Last execution start (VerticaPyLab container [Docker] time with UTC offset):</b> {current_time}"))
        vp.vDataFrame(query_val).idisplay()

button_run_query.on_click(on_button_clicked_run_query)
manual_query = widgets.VBox([query, button_run_query, output_run_query])


______

In [ ]:
footer_widget = widgets.Checkbox(
    value=True,
    description='Footer on',
    disabled=False,
    indent=False
)
count_widget = widgets.Checkbox(
    value=False,
    description='Count on',
    disabled=False,
    indent=False
)
comma_widget = widgets.Checkbox(
    value=False,
    description='Insert commas for numbers',
    disabled=False,
    indent=False
)
percent_bar_widget = widgets.Checkbox(
    value=False,
    description='Percent bar',
    disabled=False,
    indent=False
)
max_cell_width_widget = widgets.BoundedIntText(
    value=280,
    min=250,
    max=350,
    step=1,
    description='Maximum cell width:',
    disabled=False,
    layout=widgets.Layout(width='300px')
)
max_cell_width_widget.style.description_width = '240px'
max_columns_widget = widgets.BoundedIntText(
    value=50,
    min=1,
    max=150,
    step=1,
    description='Maximum number of columns to display:',
    disabled=False,
    layout=widgets.Layout(width='300px')
)
max_columns_widget.style.description_width = '240px'
max_rows_widget = widgets.BoundedIntText(
    value=100,
    min=1,
    max=4000,
    step=1,
    description='Maximum number of rows to display:',
    disabled=False,
    layout=widgets.Layout(width='300px')
)
max_rows_widget.style.description_width = '240px'
warning_text = """
<div style="background-color: #ffe5b3; color: #cc3300; padding: 10px; border: 1px solid #cc3300; border-radius: 5px;">
  <strong>Caution:</strong> These settings will reflect in your entire environment.
</div>
"""

warning_message = widgets.HTML(value=warning_text)
advanced_display_settings = widgets.VBox([
        warning_message,
        footer_widget,
        count_widget,
        comma_widget,
        percent_bar_widget,
        max_cell_width_widget,
        max_columns_widget,
        max_rows_widget,
    ],
)



update_advanced_display_options = widgets.Button(description="Update preferences")
output_advanced_display_options = widgets.Output()

def on_button_clicked_update_display_options(b):
    output_advanced_display_options.clear_output(wait=True)
    
    # Capture widget values
    with output_initial:
        global count_widget_val, footer_widget_val, comma_widget_val, percent_bar_widget_val, max_cell_width_widget_val, max_columns_widget_val, max_rows_widget_val
        count_widget_val = count_widget.value
        footer_widget_val = footer_widget.value
        comma_widget_val = comma_widget.value
        percent_bar_widget_val = percent_bar_widget.value
        max_cell_width_widget_val = max_cell_width_widget.value
        max_columns_widget_val = max_columns_widget.value
        max_rows_widget_val = max_rows_widget.value

    # Store the variables
    with output_advanced_display_options:
        %store count_widget_val footer_widget_val comma_widget_val percent_bar_widget_val max_cell_width_widget_val max_columns_widget_val max_rows_widget_val
        alert_text = """
        <div style="background-color: #e6f7ff; color: #007bff; padding: 10px; border: 1px solid #007bff; border-radius: 5px;">
          <strong>Alert:</strong> Changes have been updated. Please refresh the page to see them.
        </div>
        """
        display(widgets.HTML(value=alert_text))

update_advanced_display_options.on_click(on_button_clicked_update_display_options)
advanced_display_settings_widget = widgets.VBox([advanced_display_settings, update_advanced_display_options, output_advanced_display_options])

In [ ]:
button_run_explain_plan = widgets.Button(description="Run Explain Plan")
explain_output = widgets.Output()
try:
    explain_dropdown_options = [i for i in range(len(qprof.transactions))]
    explain_dropdown_disabled = False
except Exception as e:
    explain_dropdown_options = []
    explain_dropdown_disabled = True
    
explain_dropdown = widgets.Dropdown(
    options=explain_dropdown_options,
    description='Select Query Index',
    disabled=explain_dropdown_disabled,
    layout={'width': "300px"}
)
explain_dropdown.style.description_width = '130px'

def on_button_clicked_run_explain_plan(b):
    explain_output.clear_output()
    with explain_output:
        qprof.set_position(explain_dropdown.value)
        qprof.get_qplan_explain()


def on_button_clicked_run_query(b):
    output_run_query.clear_output(wait=True)
    global query_val, target_schema_val_2, key_val_2
    with output_run_query:
        query_val = query.value
        logging.info(f"""[Query Profile Tree Page] Running the following custom query requested by the user :{query_val}.""")
        run_query_result = vp.vDataFrame(query_val)
        display(widgets.HTML(run_query_result._repr_html_()))

button_run_explain_plan.on_click(on_button_clicked_run_explain_plan)
explain_plan_widget = widgets.VBox([explain_dropdown, button_run_explain_plan, explain_output]) 

In [ ]:
accordion = widgets.Accordion(children=[manual_query, explain_plan_widget, advanced_display_settings_widget, output_initial], titles=('Manual Queries', 'Explain Plan', 'Advanced vDataFrame Output Settings', 'Execution/Error Details'))
logging.info(f'[Query Profile Tree Page] Trying to display the error log at the bottom of the page')
accordion


In [ ]:
%%html
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Version Number</title>
    <style>
        .version {
            font-family: Arial, sans-serif;
            font-size: 14px;
            color: #555;
            padding: 5px 10px;
            background-color: #f9f9f9;
            border: 1px solid #ddd;
            border-radius: 4px;
            display: inline-block;
            margin-top: 10px;
        }
    </style>
</head>
<body>
    <div class="version">
        Page version update date: 6/24/2024
    </div>
</body>
</html>